### Instructions
1. Make sure you have changed the ```IP_ADDRESS``` and ```HTTP_SPEAKER_IP``` variables.
2. Make sure your mosquitto is running
3. Make sure Text to Speech App is running - navigate to HTTP TTS Server -> Start Background Service
4. Ideally, you should see "Connected" printed when runnning the 2nd block of code

In [1]:
import paho.mqtt.client as mqtt
import json
import os
import cv2
import numpy as np
import requests
from autocorrect import Speller
import pip._vendor.requests 

spell = Speller() # Autocorrect

global alphabet, sentence
sentence = ""
alphabet = ""

IP_ADDRESS = "192.168.43.172" # Laptop IP - obtained through ipconfig in cmd prompt
HTTP_SPEAKER_IP ="192.168.43.1:8081" # Obtained from Text To Speech App - make sure it uses a different port number from the ip camera!

USERNAME = os.getenv('_USERNAME')
PASSWORD = os.getenv('_PASSWORD')

def setup(hostname):
    client = mqtt.Client()
    client.on_connect = on_connect
    client.on_message = on_message
    client.username_pw_set(USERNAME, PASSWORD)
    client.connect(hostname)
    client.loop_start()
    return client

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected.")
        client.subscribe("Group_25/NLP/Input")
    else:
        print("Failed to Connect. Error code: %d." % rc)

        
def on_message(client, userdata, msg):
    global sentence
    if msg.payload == b'output': 
        try:
            r = requests.post(f'http://{HTTP_SPEAKER_IP}/text="{sentence}"&lang=en&vol=10', timeout = 2)
            client.publish("Group_25/NLP/Output", sentence)
        except Exception as e:
            print(f"Error sending sentence to speaker: {e}")
        try:
            for word in sentence.split():
                ### Sent output words to database
                response = pip._vendor.requests.post(f'http://{IP_ADDRESS}:5000/add/word', json={"word":word}, verify=False)
                print(response.text)
        except Exception as e:
            print(f"Unable to send to database")
        sentence = "" # reset sentence after sending to database
    else:
        modify_sentence((msg.payload).decode())
        client.publish("Group_25/NLP/Output", sentence + "_")

def modify_sentence(alphabet):
    global sentence
    if alphabet == 'del':
        sentence = sentence[:-1]
    elif alphabet == 'space':
        sentence += " "
        new_sentence = ""
        for word in sentence.split():
            original = word[:]
            if word in dict_.keys():
                word = dict_[word]
            word = spell(word) #autocorrect
            if original != word:
                try:
                    ### Sent mispelled word to database
                    response = pip._vendor.requests.post(f'http://{IP_ADDRESS}:5000/add/misspelled', json={"predicted": original, "autocorrected": word}, verify=False)
                    print(response.text)
                except Exception as e:
                    print(f"Unable to send mispelled words to database")
            
            new_sentence += word 
            new_sentence += " "
        sentence = new_sentence[:]
    else: 
        sentence += alphabet  
        
    print(f"Sentence: {sentence}")
            
def main():
    client = setup(IP_ADDRESS)

In [2]:
if __name__ == "__main__":
    ### Example of dictionary that can be used for autocorrecting
    dict_ = {
    "hf": "hi",
    "hp": "hi",
    "hj": "hi",
    "hv": "hi",
    "hd": "hi",
    "prcf": "prof",
    "prff": "prof",
    "pkof": "prof",
    "ue": "us",
    "ua": "us",
    "plue":"plus"
    }
    main()


Connected.
